<a href="https://colab.research.google.com/github/Medalcode/Colab1/blob/main/Eval3Mineria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

aqui instalamos y cargamos las librerías

tidyverse (importación, manipulación, visualización y análisis de datos)

readxl (facilita la extracción de datos de Excel a R)

pandas


In [7]:
import pandas as pds  #entonces pds es pandas
import numpy as npy   #el apodo de numpy es npy
import io             #la libreria io permite trabajar con flujos de entrada y salida de datos
import base64
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats import skew, kurtosis
import os
from io import BytesIO

In [8]:
# Configuración inicial para Matplotlib/Seaborn
sns.set_theme(style="whitegrid")

# Variable global para almacenar el DataFrame y el historial de operaciones (Log)
# Esto simula gr.State() para mantener el estado entre las llamadas a las funciones.
df_state = None
log_entries = []

In [9]:
def load_data(file_obj, delimiter_choice):
    """I.1 load_data: Carga el archivo subido en un DataFrame."""
    global df_state, log_entries
    log_entries = []

    if file_obj is None:
        return None, "Error: Debe subir un archivo.", None

    file_path = file_obj.name

    try:
        if file_path.endswith('.csv'):
            if delimiter_choice == "Comma (,)":
                delimiter = ','
            elif delimiter_choice == "Semicolon (;)":
                delimiter = ';'
            else:
                delimiter = ',' # Default fallback

            # Intentar leer el archivo con codificaciones comunes [7]
            try:
                df = pd.read_csv(file_path, delimiter=delimiter, encoding='utf-8')
            except UnicodeDecodeError:
                df = pd.read_csv(file_path, delimiter=delimiter, encoding='ISO-8859-1')

        elif file_path.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(file_path) # Requiere openpyxl [4]

        else:
            return None, "Error: Archivo no válido. Suba un archivo CSV o Excel.", None

        # Validación de tipos de datos (debe contener categóricos y numéricos) [6]
        num_cols = df.select_dtypes(include=np.number).columns
        cat_cols = df.select_dtypes(include=['object', 'category']).columns

        if len(num_cols) == 0 or len(cat_cols) == 0:
            msg = "Advertencia: El archivo debe contener tanto datos categóricos como numéricos para un análisis completo."
        else:
            msg = f"Archivo cargado. Dimensiones: {df.shape}. {len(num_cols)} numéricas, {len(cat_cols)} categóricas."

        df_state = df
        log_entries.append(f"Carga: Archivo {os.path.basename(file_path)} cargado. Se detectaron {df.shape} filas.")

        # Retorna el DataFrame, el mensaje y una vista previa [8]
        return df_state, msg, gr.Dataframe(value=df.head())

    except Exception as e:
        df_state = None
        return None, f"Error de lectura: {str(e)}", None

In [10]:
def get_numeric_cols(df):
    """Función auxiliar para obtener columnas numéricas."""
    return df.select_dtypes(include=np.number).columns.tolist()

def handle_missing_values(df_in, col_name, method):
    """II.1 handle_missing_values: Manejo interactivo de valores nulos."""
    global df_state, log_entries
    df = df_in.copy()

    # Manejar el caso donde no hay DF cargado
    if df is None:
        return None, "Error: Primero cargue un archivo.", None

    col_names = [c.strip() for c in col_name.split(",") if c.strip()]

    # Contar nulos antes de la operación [9]
    total_nulos_antes = df[col_names].isnull().sum().sum()
    registros_afectados = 0

    if total_nulos_antes == 0:
        msg = "No se encontraron valores nulos en las columnas seleccionadas. No se realizó ninguna operación."
        return df_state, msg, None

    if method == "Eliminar filas":
        filas_originales = len(df)
        df = df.dropna(subset=col_names)
        registros_afectados = filas_originales - len(df)
        log_entries.append(f"Limpieza Nulos: Se eliminaron {registros_afectados} filas con nulos en {', '.join(col_names)}.")

    else: # Métodos de imputación (promedio, máximo, mínimo, cero)
        for col in col_names:
            if col not in df.columns or col not in get_numeric_cols(df):
                log_entries.append(f"Advertencia: La columna '{col}' no es numérica o no existe para imputación.")
                continue

            if method == "Llenar con promedio":
                valor = df[col].mean()
            elif method == "Llenar con máximo":
                valor = df[col].max()
            elif method == "Llenar con mínimo":
                valor = df[col].min()
            elif method == "Llenar con cero":
                valor = 0

            df[col] = df[col].fillna(valor)
            registros_afectados += df[col].isnull().sum() # Sumará 0 si la imputación fue exitosa
            log_entries.append(f"Limpieza Nulos: La columna '{col}' se imputó con {method.split(' ')[-1]} ({valor:.2f}).")

    df_state = df # Actualizar el estado global
    msg = f"Limpieza completada. {total_nulos_antes} valores nulos tratados. Registros afectados: {registros_afectados}."
    return df_state, msg, gr.Dataframe(value=df_state.head())


def apply_scaling(df_in, col_name, method):
    """II.2 apply_scaling: Aplica normalización o estandarización."""
    global df_state, log_entries
    df = df_in.copy()

    if df is None:
        return None, "Error: Primero cargue un archivo.", None

    col_names = [c.strip() for c in col_name.split(",") if c.strip()]
    if not all(col in df.columns and col in get_numeric_cols(df) for col in col_names):
        return df_state, "Error: Verifique que las columnas existan y sean numéricas.", None

    if method == "Min-Max":
        scaler = MinMaxScaler()
        justificacion = "**Recomendación Min-Max:** Se recomienda para algoritmos que esperan un rango acotado (ej. Redes Neuronales) o cuando la distribución no es gaussiana. Sin embargo, es sensible a los *outliers* [10-12]."
    elif method == "Z-Score":
        scaler = StandardScaler()
        justificacion = "**Recomendación Z-Score:** Se recomienda para algoritmos basados en distancias (ej. K-Means, KNN) o cuando se asume una distribución aproximadamente normal. Es menos sensible a los *outliers* que Min-Max [10, 12, 13]."
    else:
        return df_state, "Método de escalado no válido.", None

    for col in col_names:
        df[col] = scaler.fit_transform(df[[col]])
        log_entries.append(f"Escalado: Columna '{col}' escalada usando {method}.")

    df_state = df
    msg = f"Escalado de {', '.join(col_names)} completado usando {method}. {justificacion}"
    return df_state, msg, gr.Dataframe(value=df_state.head())


def detect_and_treat_outliers(df_in, col_name, treatment):
    """II.3 detect_and_treat_outliers: Detección por IQR y tratamiento."""
    global df_state, log_entries
    df = df_in.copy()

    if df is None:
        return None, "Error: Primero cargue un archivo.", None

    if col_name not in df.columns or col_name not in get_numeric_cols(df):
        return df_state, f"Error: La columna '{col_name}' no existe o no es numérica.", None

    # Detección por IQR [10, 14-17]
    Q1 = df[col_name].quantile(0.25)
    Q3 = df[col_name].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    outliers_detectados = df[(df[col_name] < limite_inferior) | (df[col_name] > limite_superior)]
    n_outliers = len(outliers_detectados)

    if n_outliers == 0:
        msg = f"No se detectaron *outliers* en la columna '{col_name}' (Método IQR)."
        return df_state, msg, None

    # Tratamiento [10, 18-20]
    if treatment == "Eliminar registros":
        df = df[~((df[col_name] < limite_inferior) | (df[col_name] > limite_superior))]
        log_entries.append(f"Outliers: Se eliminaron {n_outliers} *outliers* en '{col_name}'.")
        msg = f"Se detectaron y eliminaron {n_outliers} *outliers* en '{col_name}'. Se eliminaron {n_outliers} filas."

    elif treatment == "Capping (Winsorización)":
        # Limitar valores extremos al límite inferior y superior del bigote [19]
        df[col_name] = np.where(df[col_name] > limite_superior, limite_superior, df[col_name])
        df[col_name] = np.where(df[col_name] < limite_inferior, limite_inferior, df[col_name])
        log_entries.append(f"Outliers: Se aplicó *capping* a {n_outliers} *outliers* en '{col_name}'.")
        msg = f"Se detectaron {n_outliers} *outliers* y se aplicó *Capping* (Winsorización) para conservar los registros."

    else:
        msg = f"Se detectaron {n_outliers} *outliers* en '{col_name}'. Se recomienda tratarlos, ya que pueden sesgar la media y la desviación estándar [15, 21]."
        df_state = df_in
        return df_state, msg, None

    df_state = df
    return df_state, msg, gr.Dataframe(value=df_state.head())

In [11]:
def run_analysis(df_in):
    """III.1 run_analysis: Calcula estadísticas descriptivas, correlación, curtosis y asimetría."""
    if df_in is None:
        return "Error: Primero cargue y procese el archivo.", None

    df_num = df_in.select_dtypes(include=np.number)

    if df_num.empty:
        return "El DataFrame no contiene columnas numéricas para el análisis estadístico.", None

    # 1. Estadísticas Descriptivas [25-27]
    estadisticas = df_num.describe().T

    # 2. Correlaciones [25, 27]
    correlaciones = df_num.corr(method='pearson') # Pearson para relación lineal [28]

    # 3. Curtosis y Asimetría (Skewness) [25, 27, 29, 30]
    curtosis_series = df_num.apply(kurtosis, fisher=False) # Fisher=False para valor absoluto (Normal=3)
    asimetria_series = df_num.apply(skew)

    resumen_forma = pd.DataFrame({
        'Curtosis (Normal ≈ 3)': curtosis_series,
        'Asimetría (Skewness)': asimetria_series
    }).round(3)

    # Interpretación de Resultados (Ejemplo)
    interpretacion = "### Resumen de Interpretación:\n"
    interpretacion += "- **Curtosis:** Los valores > 3 (Leptocúrtica) indican un pico más agudo y colas pesadas, sugiriendo más *outliers* [30].\n"
    interpretacion += "- **Asimetría:** Valores positivos (> 0) indican sesgo a la derecha (media > mediana) [29, 31].\n"
    interpretacion += "- **Correlación:** Los valores cercanos a 1 o -1 en el mapa de calor indican relaciones lineales fuertes entre pares de variables [32].\n"

    log_entries.append("Análisis Estadístico: Cálculos descriptivos, curtosis y asimetría generados.")

    # Formatear el resultado en un solo string
    resultado_texto = (
        f"{interpretacion}\n\n"
        f"**Estadísticas Descriptivas (Media, Desviación, Cuartiles):**\n{estadisticas.to_markdown()}\n\n"
        f"**Forma de la Distribución (Curtosis y Asimetría):**\n{resumen_forma.to_markdown()}\n"
    )

    return resultado_texto, correlaciones


def generate_plots(df_in, col_corr, col_dist):
    """III.2 generate_plots: Genera un mapa de calor y un histograma/boxplot."""
    if df_in is None:
        return None, "Error: Primero cargue el archivo.", None

    df_num = df_in.select_dtypes(include=np.number)

    # Gráfico 1: Mapa de Correlación (Heatmap) [25, 27, 33]
    plt.figure(figsize=(10, 8))
    sns.heatmap(df_num.corr(), annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Mapa de Calor de Correlaciones (Pearson)")
    correlation_plot_path = "correlation_plot.png"
    plt.savefig(correlation_plot_path)
    plt.close()

    # Gráfico 2: Distribución (Histograma + Boxplot) [25, 26]
    if col_dist in df_num.columns:
        fig, axes = plt.subplots(2, 1, figsize=(8, 8), sharex=True)

        # Histograma (Distribución) [34]
        sns.histplot(df_num[col_dist], kde=True, ax=axes)
        axes.set_title(f"Distribución de: {col_dist} (Histograma y KDE)")

        # Boxplot (Detección de Outliers) [15, 26]
        sns.boxplot(x=df_num[col_dist], ax=axes[35])
        axes[35].set_title(f"Boxplot de: {col_dist} (Outliers: 1.5*IQR)")

        plt.tight_layout()
        distribution_plot_path = "distribution_plot.png"
        plt.savefig(distribution_plot_path)
        plt.close()
    else:
        distribution_plot_path = None
        log_entries.append("Advertencia: No se pudo generar el gráfico de distribución, columna no numérica o inexistente.")

    log_entries.append("Visualización: Gráficos de correlación y distribución generados.")
    return correlation_plot_path, distribution_plot_path


def export_results(df_in, export_format):
    """IV.1 export_results: Permite la exportación y genera el reporte de log."""
    global log_entries

    if df_in is None:
        return "Error: No hay datos procesados para exportar.", None, None

    # --- 1. Generar el reporte breve automático (Log) [24, 36, 37]
    reporte_path = "reporte_analisis.txt"
    log_content = "\n".join(log_entries)

    # Incluir el reporte descriptivo de la actividad [37]
    reporte_final = (
        "### REPORTE BREVE AUTOMÁTICO DE PROCESAMIENTO DE DATOS\n\n"
        "**Proceso Seguido y Decisiones Tomadas en Limpieza de Datos:**\n"
        f"{log_content}\n\n"
        f"**Interpretación Preliminar de Resultados Obtenidos:**\n"
        f"(La interpretación completa de correlaciones, curtosis y regresiones debe realizarla el analista.)\n"
        f"Se recomienda revisar el *heatmap* para correlaciones fuertes (Pearson > 0.7 o < -0.7) [32, 38].\n"
        f"La limpieza de datos asegura la calidad y reduce el sesgo en fases de modelado posteriores (GIGO: *Garbage In, Garbage Out*) [39].\n"
        f"Dimensiones del DataFrame final: {df_in.shape}\n"
    )

    with open(reporte_path, "w") as f:
        f.write(reporte_final)

    # --- 2. Exportación de datos procesados [24, 27, 36]
    if export_format == "CSV":
        salida_path = "datos_procesados.csv"
        df_in.to_csv(salida_path, index=False)
    elif export_format == "Excel":
        salida_path = "datos_procesados.xlsx"
        df_in.to_excel(salida_path, index=False)
    else:
        return "Error: Formato de exportación no válido.", None, None

    log_entries.append(f"Exportación: Datos procesados guardados en {salida_path} y Log generado.")

    return f"Exportación exitosa. Descargue el archivo y el reporte.", salida_path, reporte_path

In [12]:
# Mapear las funciones para Gradio
# Usaremos un gr.State para manejar el DataFrame a lo largo de las funciones.

def get_col_names(df_state):
    """Función que devuelve nombres de columnas numéricas para los Dropdowns."""
    if df_state is not None:
        # Devuelve las columnas numéricas para las opciones de escalado/outliers
        return df_state.select_dtypes(include=np.number).columns.tolist()
    return []

# ----------------- Estructura de la Interfaz -----------------

with gr.Blocks(title="Aplicación de Minería de Datos y EDA") as interfaz:
    gr.Markdown("## 🛠️ Aplicación Interactiva para Procesamiento y Análisis de Datos")

    # Estado para mantener el DataFrame entre llamadas
    df_state_var = gr.State(None)

    with gr.Tab("1. Carga de Datos"):
        gr.Markdown("### Carga y Validación del Archivo")
        with gr.Row():
            separador = gr.Radio(
                choices=["Comma (,)", "Semicolon (;)"],
                label="Selecciona el Separador del Archivo",
                value="Comma (,)",
                interactive=True
            )
            archivo = gr.File(label="Subir Archivo (CSV o Excel)", interactive=True)

        btn_cargar = gr.Button("Cargar y Validar")
        msg_carga = gr.Textbox(label="Mensaje de Carga")
        df_preview = gr.Dataframe(label="Vista Previa (5 primeras filas)")

        # Enlace de carga
        btn_cargar.click(
            fn=load_data,
            inputs=[archivo, separador],
            outputs=[df_state_var, msg_carga, df_preview]
        )

    with gr.Tab("2. Procesamiento y Limpieza (Data Preparation)"):
        gr.Markdown("### Limpieza de Valores Nulos")
        with gr.Row():
            metodo_nulos = gr.Radio(
                choices=["Eliminar filas", "Llenar con promedio", "Llenar con máximo", "Llenar con mínimo", "Llenar con cero"],
                label="Método para manejar nulos [6, 41]",
                value="Eliminar filas"
            )
            col_nulos = gr.Textbox(label="Columnas para Limpieza (Separadas por comas)", placeholder="Ej: Col1, Col2 (Dejar vacío para todo el DF)")

        btn_nulos = gr.Button("Aplicar Limpieza de Nulos")
        msg_nulos = gr.Textbox(label="Resultado Nulos")

        gr.Markdown("### Normalización y Estandarización")
        with gr.Row():
            metodo_normalizacion = gr.Radio(
                choices=["Min-Max", "Z-Score"],
                label="Método de Escalado [10, 42]",
                value="Z-Score"
            )
            col_normalizar = gr.Textbox(label="Columnas Numéricas para Escalar (Separadas por comas)")

        btn_normalizar = gr.Button("Aplicar Normalización / Estandarización")
        msg_normalizar = gr.Textbox(label="Resultado Normalización y Justificación [10]")

        gr.Markdown("### Detección y Tratamiento de Outliers (IQR)")
        with gr.Row():
            col_outliers = gr.Textbox(label="Columna para Detección de Outliers (Una sola columna)")
            tratamiento_outliers = gr.Radio(
                choices=["Informar", "Eliminar registros", "Capping (Winsorización)"],
                label="Tratamiento de Outliers [10, 19, 20]",
                value="Informar"
            )

        btn_outliers = gr.Button("Detectar y Tratar Outliers")
        msg_outliers = gr.Textbox(label="Resultado Outliers")

        # Enlaces de procesamiento
        btn_nulos.click(
            fn=handle_missing_values,
            inputs=[df_state_var, col_nulos, metodo_nulos],
            outputs=[df_state_var, msg_nulos, df_preview]
        )
        btn_normalizar.click(
            fn=apply_scaling,
            inputs=[df_state_var, col_normalizar, metodo_normalizacion],
            outputs=[df_state_var, msg_normalizar, df_preview]
        )
        btn_outliers.click(
            fn=detect_and_treat_outliers,
            inputs=[df_state_var, col_outliers, tratamiento_outliers],
            outputs=[df_state_var, msg_outliers, df_preview]
        )


    with gr.Tab("3. Análisis y Visualización"):
        gr.Markdown("### Análisis Estadístico (Correlación, Curtosis y Asimetría) [25]")

        btn_analisis = gr.Button("Ejecutar Análisis Estadístico")
        analisis_output = gr.Markdown(label="Resumen Estadístico e Interpretación")

        gr.Markdown("### Visualización de Datos Procesados [25]")
        with gr.Row():
            col_distribucion = gr.Textbox(label="Columna para Gráfico de Distribución (Histograma/Boxplot)")

        btn_graficos = gr.Button("Generar Gráficos")

        with gr.Row():
            plot_corr = gr.Plot(label="Mapa de Calor de Correlaciones")
            plot_dist = gr.Plot(label="Distribución y Outliers (Boxplot/Histograma)")

        # Enlaces de análisis
        btn_analisis.click(
            fn=run_analysis,
            inputs=[df_state_var],
            outputs=[analisis_output, gr.State(None)] # El segundo output no es usado pero necesario para que el DF se mantenga en el estado
        )
        btn_graficos.click(
            fn=generate_plots,
            inputs=[df_state_var, gr.State(None), col_distribucion],
            outputs=[plot_corr, plot_dist]
        )


    with gr.Tab("4. Exportación y Reporte"):
        gr.Markdown("### Exportar Datos Procesados y Generar Log [36]")

        export_format = gr.Radio(
            choices=["CSV", "Excel"],
            label="Seleccionar Formato de Exportación",
            value="CSV"
        )

        btn_exportar = gr.Button("Generar Archivos Finales")
        msg_export = gr.Textbox(label="Resultado de la Exportación")

        # Los archivos de descarga deben ser componentes File [43]
        file_output = gr.File(label="Descargar Datos Procesados")
        log_output = gr.File(label="Descargar Reporte de Log")

        # Enlace de exportación
        btn_exportar.click(
            fn=export_results,
            inputs=[df_state_var, export_format],
            outputs=[msg_export, file_output, log_output]
        )


# Iniciar la interfaz
if __name__ == "__main__":
    interfaz.launch(inline=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6e7c56c3538dfeb72d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
